In [1]:
import pandas as pd
import pandasql as psql
import pyodbc
import glob, os

In [2]:
cnx = pyodbc.connect('Driver={SQL Server};Server=.;Database=JNR;Trusted_Conneciton=Yes')

In [3]:
path = 'C:\\Users\\Juan Giron\\Documents\\git_repo\\Inventory\\'

In [4]:
# read the entire directory and concat to a df
all_files = glob.glob(os.path.join(path , "*.xlsx"))

li = []

for filename in all_files:
    df = pd.read_excel(filename)
    df['OBJECT_NAME'] = filename.replace(path+"\\","")
    df['OBJECT_NAME'] = filename.replace(path,"")
    df['TOTAL'] = df['TOTAL'].astype(int)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame

,DESCRIPTION,SKU,TOTAL,OBJECT_NAME
0,Ascent - 50ml EDP Romantic Magnolia,857792005238,28,3PL_GROUP.xlsx
1,Kopari - 2oz/(60ml) Detox Mask,851989006763,921,3PL_GROUP.xlsx
2,Kopari - Coco Mint Oil Pulling - Cylinder,851989006701,717,3PL_GROUP.xlsx
3,Kopari - Love Melt Carton - 15 Packettes,851989006961,1889,3PL_GROUP.xlsx
4,Kopari - 3.4oz/(96g) Coconut Charcoal Toothpaste,851989006794,1756,3PL_GROUP.xlsx
...,...,...,...,...
4481,627843736677 (Voir Haircare - Rhythm of the Ra...,627843736677,8528,RJTB_GROUP_QBW.xlsx
4482,883967471119 (Catherines Garden 2 Wick Lace Ed...,883967471119,108,RJTB_GROUP_QBW.xlsx
4483,883967471324 (Caherines Garden Gift Bosed Asso...,883967471324,1027,RJTB_GROUP_QBW.xlsx
4484,857798225849 (Mario Lopez Red Deo Spray 2-Pack...,857798225849,-1,RJTB_GROUP_QBW.xlsx


In [10]:
frame = psql.sqldf('''
select
DESCRIPTION,
SKU,
cast(TOTAL as int) as TOTAL,
OBJECT_NAME
from frame
''')

In [11]:
cursor = cnx.cursor()
# Insert Dataframe into SQL Server:
for index, row in frame.iterrows():
     cursor.execute("INSERT INTO dbo.tblRJTB_INVENTORY_SOURCE values(?,?,?,?)", row.DESCRIPTION,row.SKU,row.TOTAL,row.OBJECT_NAME)
cnx.commit()
cursor.close()